In [1]:
import pandas as pd
import numpy as np
import math
import random
from itertools import combinations
import time
import matplotlib.pyplot as plt
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows

df = pd.read_csv('pjt_data/world_Cup_2022_db_11_18_upd.csv')
df

,rank_Local,rank_Global,team_Name,rating,average_Rank,average_Rating,group_Name,distance(default = from Quatar,Unnamed: 8,Continental,GDP(2022),"Population(2022,per 10,000)",star_Player,1year_Change_Rank,1year_Change_Rating
0,1,1,Brazil,2169,4,1999,group_G,11974.67?km,13.12,America,"8,857","21,399","Gabriel Jesus,Gabriel Martinelli,Vinicius junior",0,20
1,2,2,Argentina,2143,5,1984,group_C,13817.43km,15.14,America,"13,621","4,560",NaN,1,42
2,3,3,Spain,2045,7,1941,group_E,5304.62km,5.81,Europe,"29,198","4,674","Iago aspas,Joselu,Mikel merino,Muniain,Sergio ...",2,8
3,4,4,Netherlands,2040,15,1844,group_A,4848.23km,5.31,Europe,"56,297","1,740",NaN,8,111
4,5,5,Belgium,2025,23,1751,group_F,4844.61km,5.31,Europe,"50,597","1,187","Kevin de Bruyne,Thibaut courtois",-1,-50
5,6,6,France,2005,16,1784,group_D,4889.93?km,5.36,Europe,"42,330","6,542","William Saliba,Karim benzema,Nabil fekir,Nkunk...",-4,-110
6,7,7,Portugal,2004,19,1789,group_H,5688.18?km,6.23,Europe,"24,910","1,016","Joao Cancelo,Andre silva",1,31
7,8,9,Denmark,1971,20,1798,group_D,4785.08km,5.24,Europe,"65,713",581,NaN,1,34
8,9,10,Germany,1963,8,1910,group_E,4477.97km,4.91,Europe,"48,397","8,431","Gnabry,Jonas hofmann,Jonathan burkhardt,Polter",-1,0
9,10,11,Uruguay,1936,12,1878,group_H,12997.55km,14.24,America,"20,017",340,NaN,6,88


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   rank_Local                      32 non-null     int64  
 1   rank_Global                     32 non-null     int64  
 2   team_Name                       32 non-null     object 
 3   rating                          32 non-null     int64  
 4   average_Rank                    32 non-null     int64  
 5   average_Rating                  32 non-null     int64  
 6   group_Name                      32 non-null     object 
 7   distance(default = from Quatar  32 non-null     object 
 8   Unnamed: 8                      32 non-null     float64
 9   Continental                     32 non-null     object 
 10  GDP(2022)                       32 non-null     object 
 11  Population(2022,per 10,000)     32 non-null     object 
 12  star_Player                     9 non-

In [3]:
def rating_Upd(team_A,team_B,win_Rate_A,df_group_Alphabet) :
    
    before_team_A_rating=df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'rating'].values
    before_team_B_rating=df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'rating'].values
    
    df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'rating'] = int(df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'rating'].values) + int(24*(1-(win_Rate_A))) # 32 는 k값임
    
    df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'rating'] = int(df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'rating'].values) + int(24*(0-(1-win_Rate_A)))  # k = 32,24,16 등 승부가 낫을 때, k값이 클수록 변동이 커짐
    
    print(f"""
    승리한 {team_A} 의 rating :{df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'rating'].values[0]} (+{df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'rating'].values[0]-before_team_A_rating[0]})
    패배한 {team_B} 의 rating :{df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'rating'].values[0]} ({df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'rating'].values[0]-before_team_B_rating[0]})
    """)   
def run_One_Game(team_A,team_B,rating_A,rating_B,df_group_Alphabet,n):

    win_Rate_A = 1 / (1 + 10**( (rating_B-rating_A) / 400) )
    
    num_Rand = random.random()
    
    print("------------------------------------")
    print(f"""
    {df_group_Alphabet['group_Name'].unique()} 조 {n+1}번째 경기시작
    {team_A} vs {team_B}
    
    {team_A}의 승리 확률 : {win_Rate_A*100}
    {team_B}의 승리 확률 : {(1-win_Rate_A)*100}
    
    {team_A}의 rating : {df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'rating'].values}
    {team_B}의 rating : {df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'rating'].values}
    
    난수 : {num_Rand}
    
    """)
    
    if num_Rand <= win_Rate_A:
        print(f"""
        {team_A} (이)가 승리했습니다.
        {team_B} (이)가 패배했습니다.
        """)
        df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'win'] += 1
        df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'point'] += 3
        df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'lose'] += 1
        
        rating_Upd(team_A,team_B,win_Rate_A,df_group_Alphabet)
        
    else :
        print(f"""
        {team_B} (이)가 승리했습니다.
        {team_A} (이)가 패배했습니다.
        """)
        
        df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_A,'lose'] += 1
        df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'win'] += 1
        df_group_Alphabet.loc[df_group_Alphabet['team_Name']==team_B,'point'] += 3
        
        rating_Upd(team_B,team_A,1-win_Rate_A,df_group_Alphabet)
    
    
    # run_One_Game 결과 dataFrame으로 출력하기 (수정중)
    
    # series_Run_One_Game_Result = df_group_Alphabet[(df_group_Alphabet['team_Name'] == team_A) | (df_group_Alphabet['team_Name'] == team_B)]['rating']
    # df_Run_One_Game_Result = pd.DataFrame(series_Run_One_Game_Result,columns=['win_Rate'])
    
    # df_Run_One_Game_Result.set_index(keys=[team_A,team_B])
    
    # df[(df['country']=='한국') | (df['country']=='호주')]
    
    # df_Group_A = df[df['group_Name']==all_Group_Name[i]][['group_Name','team_Name','rating']].reset_index(drop=True)
    # print(type(df_Run_One_Game_Result))
    # print(df_Run_One_Game_Result)
    # df[       df['group_Name']==all_Group_Name[i]     ][['group_Name','team_Name','rating']].reset_index(drop=True)
    
    print("------------------------------------")
    

def run_Group_Games(df_group_Alphabet): # df_Group_A 이라는 Datafrme 파라미터로 받음
    
    print("-------------------------------------------------------------------------------")
    print(f"{df_group_Alphabet['group_Name'].unique()} 조 모든 경기 시작") 
    group_Teams = [team_Name for team_Name in df_group_Alphabet['team_Name']] 
    # ['Netherlands', 'Ecuador', 'Senegal', 'Qatar']
    
    group_games = list(combinations(group_Teams, 2)) 
    # [('Netherlands', 'Ecuador'), ('Netherlands', 'Senegal'), ('Netherlands', 'Qatar'), ('Ecuador', 'Senegal'), ('Ecuador', 'Qatar'), ('Senegal', 'Qatar')]
    
    for i in range(len(group_games)):

        run_One_Game(group_games[i][0],
                     group_games[i][1],
                     df_group_Alphabet.loc[df_group_Alphabet['team_Name']==group_games[i][0],'rating'].values,
                     df_group_Alphabet.loc[df_group_Alphabet['team_Name']==group_games[i][1],'rating'].values,
                     df_group_Alphabet,
                     i)
        
    print(f"{df_group_Alphabet['group_Name'].unique()} 조 모든 경기 종료")
    print("-------------------------------------------------------------------------------")

    
def run_All_Group_Games(all_Group_Name):
    print("============================================= 2022 World Cup 조별예선이 시작되었습니다 ============================================= ")
    for i in range(len(all_Group_Name)):
        run_Group_Games(all_Group_Name[i]) # df_Group_A 
    print("=============================================  2022 World Cup 조별예선이 종료되었습니다 ============================================= ")
    
def result_All_Group_Games(all_Group_Df):
    
    df_Result_One_Worldcup = pd.DataFrame(index=['result1'])
    #df_Result_One_Worldcup = pd.DataFrame(columns=[ j for j in i['team_Name'] for i in all_Group_Df] ) # all_Group_Df = [df_Group_A,df_Group_B....]
    
    for i in all_Group_Df:
        for j in i['team_Name']:
            df_Result_One_Worldcup[[j + '_seed1', j + '_seed2']] = [0,0]


    for i in range(len(all_Group_Df)):
        all_Group_Df[i] = all_Group_Df[i].sort_values(by=['point', 'rating'] ,ascending=False)
        all_Group_Df[i] = all_Group_Df[i].reset_index(drop=True)
        all_Group_Df[i].index += 1
        all_Group_Df[i].loc[1,'seed'] = 'seed1'
        all_Group_Df[i].loc[2,'seed'] = 'seed2'
        if all_Group_Df[i].loc[1,'seed'] == 'seed1':
            df_Result_One_Worldcup[all_Group_Df[i].loc[1,'team_Name'] + '_seed1'] = 1
        if all_Group_Df[i].loc[2,'seed'] == 'seed2':
            df_Result_One_Worldcup[all_Group_Df[i].loc[2,'team_Name'] + '_seed2'] = 1
        
        print(f"{all_Group_Df[i]}\n")
    
    #https://wooiljeong.github.io/python/openpyxl-pandas/
#     wb = openpyxl.Workbook()
#     ws = wb.active
#     for r in dataframe_to_rows(df_Result_One_Worldcup, index=False, header=True):
#         ws.append(r)
    
    df_Result_One_Worldcup.to_excel('df_Result_One_Worldcup.xlsx')

def make_All_Groups():
    all_Group_Name = [group_Name for group_Name in df['group_Name'].unique()]
    all_Group_Name.sort() # ['group_A', 'group_B', 'group_C', 'group_D', 'group_E', 'group_F', 'group_G', 'group_H']
    
    all_Group_Df = ['df_Group_'+chr(65+i) for i in range(len(all_Group_Name))]

    for i in range(len(all_Group_Name)) : # i = 0,1,2,3,4,5,6,7                #range(1,int(df['team_Name'].count()/4)+1) :# i = 1,2,3,4,5,6,7,8
    
        all_Group_Df[i] = df[df['group_Name']==all_Group_Name[i]][['group_Name','team_Name','rating']].reset_index(drop=True)
        all_Group_Df[i].index += 1
        all_Group_Df[i][['win','draw','lose','point','seed']] = [0,0,0,0,'-']
        print(f"{all_Group_Df[i]}\n")

    return all_Group_Df


In [4]:
# Main 문

start = time.time()

all_Group_Df = make_All_Groups() # 월드컵 조편성 !
run_All_Group_Games(all_Group_Df) # 월드컵 시작 !
result_All_Group_Games(all_Group_Df) # 월드컵 결과 !

print(f"함수 실행 시간 :{time.time() - start}",)

  group_Name    team_Name  rating  win  draw  lose  point seed
1    group_A  Netherlands    2040    0     0     0      0    -
2    group_A      Ecuador    1833    0     0     0      0    -
3    group_A      Senegal    1687    0     0     0      0    -
4    group_A        Qatar    1680    0     0     0      0    -

  group_Name      team_Name  rating  win  draw  lose  point seed
1    group_B        England    1920    0     0     0      0    -
2    group_B  United States    1798    0     0     0      0    -
3    group_B           Iran    1797    0     0     0      0    -
4    group_B          Wales    1790    0     0     0      0    -

  group_Name     team_Name  rating  win  draw  lose  point seed
1    group_C     Argentina    2143    0     0     0      0    -
2    group_C        Poland    1814    0     0     0      0    -
3    group_C        Mexico    1809    0     0     0      0    -
4    group_C  Saudi Arabia    1635    0     0     0      0    -

  group_Name  team_Name  rating  win 